# とりあえず使ってみよう！

natto-py という MeCab バインディングを簡単に紹介します。

### 必要な環境

1. Python 2.7 あるいはそれ以上なもの
2. MeCab 0.996
3. 上記 MeCab 0.996 の利用可能なシステム辞書（以下は mecab-ipadic の利用を想定する）

※　このノートブックの文字コードが UTF-8 であるため、mecab の charset を UTF-8 に設定する必要があります。詳しくは[文字コード変更](http://taku910.github.io/mecab/#charset)を参照してください。

### インストール

インストールするには `pip` を通して通常のパッケージと同様にインストールを行います。    
    
    pip install natto-py


### 設定

下記の条件さえ揃っていれば、特にあらかじめに設定する必要はなく、natto-py は自動的に mecab ライブラリーを特定し、バインディングをします。

- \*NIX と OSX の場合い、mecab および mecab-config を PATH に追加していること
- また Windows では、通常のインストーラーを使用して Windows Registry に `HKEY_CURRENT_USER\Software\MeCab` というキーが設定していること

万が一、mecab ライブラリーが見つからず `EnvironmentError` などが発生した場合、次の環境変数設定を行う必要があります。

- mecab システム辞書の charset を `MECAB_CHARSET=utf8` とする
- mecab ライブラリーのファイル名 を `MECAB_PATH` とする

### 基本の使い方

#### `import` 文

`natto` から `MeCab` クラスをインポートしてインスタンスを取得します。

`print` でインスタンスの詳細情報が確認できます。

In [1]:
from natto import MeCab

nm = MeCab()

print(nm)

<natto.mecab.MeCab model=<cdata 'mecab_model_t *' 0x104b397e0>, tagger=<cdata 'mecab_t *' 0x104b104d0>, lattice=<cdata 'mecab_lattice_t *' 0x104b23000>, libpath="/usr/local/Cellar/mecab/0.996/lib/libmecab.dylib", options={}, dicts=[<natto.dictionary.DictionaryInfo dictionary=<cdata 'mecab_dictionary_info_t *' 0x104b13ee0>, filepath="/usr/local/Cellar/mecab/0.996/lib/mecab/dic/ipadic/sys.dic", charset=utf8, type=0>], version=0.996>


#### 解析結果を標準出力に

文をとりあえず解析して結果を文字列として標準出力に送ります。

In [2]:
text = "ピンチの時には必ずヒーローが現れる。"

print(nm.parse(text))

ピンチ	名詞,一般,*,*,*,*,ピンチ,ピンチ,ピンチ
の	助詞,連体化,*,*,*,*,の,ノ,ノ
時	名詞,非自立,副詞可能,*,*,*,時,トキ,トキ
に	助詞,格助詞,一般,*,*,*,に,ニ,ニ
は	助詞,係助詞,*,*,*,*,は,ハ,ワ
必ず	副詞,助詞類接続,*,*,*,*,必ず,カナラズ,カナラズ
ヒーロー	名詞,一般,*,*,*,*,ヒーロー,ヒーロー,ヒーロー
が	助詞,格助詞,一般,*,*,*,が,ガ,ガ
現れる	動詞,自立,*,*,一段,基本形,現れる,アラワレル,アラワレル
。	記号,句点,*,*,*,*,。,。,。
EOS


#### with 文

[Python with](http://docs.python.jp/3/reference/compound_stmts.html#the-with-statement) 文を使用すれば、コンテキストが正常に終了しても例外が起きても `MeCab` ライブラリーの参照が自動的に破棄されるので、勧めます。

In [3]:
with MeCab() as nm:
    print(nm.parse(text))

ピンチ	名詞,一般,*,*,*,*,ピンチ,ピンチ,ピンチ
の	助詞,連体化,*,*,*,*,の,ノ,ノ
時	名詞,非自立,副詞可能,*,*,*,時,トキ,トキ
に	助詞,格助詞,一般,*,*,*,に,ニ,ニ
は	助詞,係助詞,*,*,*,*,は,ハ,ワ
必ず	副詞,助詞類接続,*,*,*,*,必ず,カナラズ,カナラズ
ヒーロー	名詞,一般,*,*,*,*,ヒーロー,ヒーロー,ヒーロー
が	助詞,格助詞,一般,*,*,*,が,ガ,ガ
現れる	動詞,自立,*,*,一段,基本形,現れる,アラワレル,アラワレル
。	記号,句点,*,*,*,*,。,。,。
EOS


#### MeCabNode と出力フォーマット

解析結果を `MeCabNode` で取得して各形態素に関するより詳しい情報を出力します。

`MeCab` の `-F / --node-format` オプションを利用して特定の形態素情報を指定して csv 形式の出力なども可能です。

In [4]:
# -F / --node-format オプションでノードの出力フォーマットを指定する
#
# %m    ... 形態素の表層文
# %f[0] ... 品詞
# %h    ... 品詞 ID (IPADIC)
# %f[8] ... 発音
#  
with MeCab(r"-F%m,%f[0],%h,%f[8]") as nm:
    for n in nm.parse(text, as_nodes=True):
        print(n.feature)

ピンチ,名詞,38,ピンチ
の,助詞,24,ノ
時,名詞,66,トキ
に,助詞,13,ニ
は,助詞,16,ワ
必ず,副詞,35,カナラズ
ヒーロー,名詞,38,ヒーロー
が,助詞,13,ガ
現れる,動詞,31,アラワレル
。,記号,7,。
EOS


以上。